In [4]:
using Pkg
Pkg.activate("..")

  Activating project at `~/Desktop/MIT_research/Projects/mosquito/final_paper/MosquitoProject`


In [5]:
using MosquitoProject
using JLD
using CSV, DataFrames

In [8]:
mosquito_data = "../data/5-1-23-Person-Regular-Clothes"

"../data/5-1-23-Person-Regular-Clothes"

In [9]:
if isfile(mosquito_data * "_clean_traces.jld")
    println("Loading existing clean traces file: ", mosquito_data[1:end-4] * "_clean_traces.jld")
    starting_times, clean_traces = load(clean_traces_file, "clean_traces")
else
    println("Clean traces file not found. Generating new clean traces...")
    starting_times, clean_traces = export_clean_traces(mosquito_data;
        min_track_length = 1.0, min_start_time = 300.0, max_start_time=Inf,
        wall_buffer_x = 0.0, wall_buffer_y = 0.0, wall_buffer_z = 0.0
    )
end

Clean traces file not found. Generating new clean traces...
Wall x position: (-84.3507871, 87.80854240000001)
Wall y position: (-103.47304805, 109.87527872000005)
Wall z position: (539.5967128999999, 784.2310280000006)


(Any[301.591, 301.601, 301.921, 302.011, 302.701, 306.401, 308.441, 308.511, 308.661, 308.801  …  596.843, 597.133, 597.513, 597.963, 600.103, 600.343, 600.823, 601.653, 602.073, 602.303], [[-0.29498062 0.995137 … 0.0 1.0; -0.30138042 0.9958304 … 0.01 1.0; … ; -0.8275552 0.40046256999999996 … 2.48 1.0; -0.8296641 0.39632732 … 2.49 1.0], [0.30272943 0.5931425 … 0.0 1.0; 0.30410595 0.5959008 … 0.009999999999999998 1.0; … ; 0.11297283999999999 0.643336975 … 1.91 0.0; 0.11064497 0.6423730500000001 … 1.92 1.0], [-0.0009048443 0.75153885 … 0.0 1.0; -0.0021575811999999996 0.7571724000000001 … 0.009999999999999998 1.0; … ; 0.091796405 0.71855367 … 1.45 0.0; 0.09137219 0.7177151 … 1.46 1.0], [0.47001637 -0.05876782999999999 … 0.0 1.0; 0.4685295 -0.06281679999999999 … 0.01 1.0; … ; 0.24782751 -0.20571615000000001 … 3.14 1.0; 0.24127905 -0.21067442 … 3.15 1.0], [-0.13645585999999998 0.4784908 … 0.0 1.0; -0.13679244000000002 0.4783737 … 0.01 0.0; … ; -0.17918427 0.52187725 … 2.58 1.0; -0.17250349 

In [11]:
clean_traces[1]

250×5 Matrix{Float64}:
 -0.294981  0.995137  6.59938  0.0   1.0
 -0.30138   0.99583   6.59984  0.01  1.0
 -0.307907  0.996642  6.60051  0.02  1.0
 -0.314456  0.997358  6.6005   0.03  1.0
 -0.320847  0.998304  6.60202  0.04  1.0
 -0.32705   0.999053  6.60216  0.05  1.0
 -0.332789  0.999665  6.60175  0.06  1.0
 -0.337926  1.0003    6.60232  0.07  1.0
 -0.339654  1.00055   6.60304  0.08  0.0
 -0.341383  1.0008    6.60374  0.09  0.0
  ⋮                                  
 -0.813986  0.431287  7.13997  2.41  1.0
 -0.816002  0.426651  7.14366  2.42  1.0
 -0.818188  0.42209   7.14845  2.43  1.0
 -0.820222  0.417388  7.15183  2.44  1.0
 -0.822281  0.413016  7.15681  2.45  1.0
 -0.824073  0.408903  7.1611   2.46  1.0
 -0.825563  0.404609  7.16258  2.47  1.0
 -0.827555  0.400463  7.1685   2.48  1.0
 -0.829664  0.396327  7.17608  2.49  1.0